In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Explore data

In [2]:
train_data = pd.read_csv("input/nlp-getting-started/train.csv")
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [13]:
train_data.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


## Train data with keyword

In [10]:
train_data_with_keyword = train_data[~train_data['keyword'].isna()]
train_data_with_keyword

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [11]:
train_data_with_keyword_target_1 = train_data_with_keyword.loc[train_data_with_keyword.target ==1]
train_data_with_keyword_target_0 = train_data_with_keyword.loc[train_data_with_keyword.target ==0]
train_data_with_keyword_target_1

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,1
46,66,ablaze,"GREENSBORO,NORTH CAROLINA",How the West was burned: Thousands of wildfire...,1
...,...,...,...,...,...
7542,10783,wreckage,"Xi'an, China",Wreckage 'conclusively confirmed' as from miss...,1
7543,10784,wreckage,Mumbai,Wreckage 'Conclusively Confirmed' as From MH37...,1
7552,10795,wrecked,"Santa Cruz, CA",Israel wrecked my home. Now it wants my land. ...,1
7559,10808,wrecked,NaN,300K exotic car wrecked in train accident \nht...,1


In [12]:
train_data_with_keyword_target_0

,id,keyword,location,text,target
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


## Train data with location

In [19]:
train_data_with_location = train_data[~train_data['location'].isna()]
train_data_with_location["location"].to_csv("locations.csv")

In [14]:
train_data_with_location_target_1 = train_data_with_location.loc[train_data_with_keyword.target ==1]
train_data_with_location_target_0 = train_data_with_location.loc[train_data_with_keyword.target ==0]

In [15]:
train_data_with_location_target_1

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1
46,66,ablaze,"GREENSBORO,NORTH CAROLINA",How the West was burned: Thousands of wildfire...,1
50,73,ablaze,"Sheffield Township, Ohio",Deputies: Man shot before Brighton home set ab...,1
...,...,...,...,...,...
7541,10782,wreckage,"New Delhi,India",Wreckage 'Conclusively Confirmed' as From MH37...,1
7542,10783,wreckage,"Xi'an, China",Wreckage 'conclusively confirmed' as from miss...,1
7543,10784,wreckage,Mumbai,Wreckage 'Conclusively Confirmed' as From MH37...,1
7552,10795,wrecked,"Santa Cruz, CA",Israel wrecked my home. Now it wants my land. ...,1


In [18]:
train_data_with_location_target_0

,id,keyword,location,text,target
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


## Train data with location and keyword

In [19]:
train_data_with_location_keyword = train_data_with_location[~train_data_with_location['keyword'].isna()]
train_data_with_location_keyword

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


* **Note:** nombre of len(train_data_with_location_keyword) == len(train_data_with_location)

=> We determine that a line with location has a also a keyword

In [20]:
africa_target_1 = train_data_with_location_target_1.loc[train_data_with_location_target_1.location=="AFRICA"]
africa_target_1

,id,keyword,location,text,target
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1


### Catch all possible location for target_1 in train data

In [21]:
target_1 = train_data.loc[train_data.target==1]
target_1

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [22]:
location_for_target_1 = target_1.location.unique()
location_for_target_1 = location_for_target_1.tolist()
print(len(location_for_target_1))
location_for_target_1[:20]

1514


[nan,
 'Birmingham',
 'AFRICA',
 'World Wide!!',
 'GREENSBORO,NORTH CAROLINA',
 'Sheffield Township, Ohio',
 'India',
 'Anaheim',
 'USA',
 'South Africa',
 'Edmonton, Alberta - Treaty 6',
 'Concord, CA',
 'Nashville, TN',
 'Santa Clara, CA',
 'Walker County, Alabama',
 'Australia',
 'North Carolina',
 'San Mateo County, CA',
 'Njoro, Kenya',
 "Your Sister's Bedroom"]

In [23]:
location_for_target_1 = list(map(str,location_for_target_1))
location_for_target_1.sort()
location_for_target_1[:20]

['  ',
 '  News',
 ' 616 \x89Û¢ Kentwood , MI ',
 ' ? ??????? ? ( ?? å¡ ? ? ? å¡)',
 ' Alberta',
 ' Baku & Erzurum ',
 ' Bouvet Island',
 ' Nevada Carson City,Freeman St',
 ' New England',
 ' Queensland, Australia',
 ' Road to the Billionaires Club',
 ' The World',
 '#1 Vacation Destination,HAWAII',
 '#????? Libya#',
 '#BlackLivesMatter',
 '#Bummerville otw',
 '#KaumElite;#F?VOR;#SMOFC',
 '#WashingtonState #Seattle',
 '#goingdownthetoilet Illinois',
 '#iminchina']

### Catch all possible keyword for target_1 in train data

In [24]:
keyword_for_target_1 = target_1.keyword.unique()
keyword_for_target_1 = keyword_for_target_1.tolist()
print(len(keyword_for_target_1))
keyword_for_target_1[:20]

221


[nan,
 'ablaze',
 'accident',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing']

In [25]:
keyword_for_target_1 = list(map(str,keyword_for_target_1))
keyword_for_target_1.sort()
keyword_for_target_1[:20]

['ablaze',
 'accident',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding']

In [26]:
target_0 = train_data.loc[train_data.target==0]
target_0

,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0
17,25,NaN,NaN,Summer is lovely,0
18,26,NaN,NaN,My car is so fast,0
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0
...,...,...,...,...,...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0
7582,10834,wrecked,NaN,Cramer: Iger's 3 words that wrecked Disney's s...,0
7584,10837,NaN,NaN,These boxes are ready to explode! Exploding Ki...,0
7587,10841,NaN,NaN,Sirens everywhere!,0


### Catch all possible location for target_0 in train data

In [27]:
location_for_target_0 = target_0.location.unique()
location_for_target_0 = location_for_target_0.tolist()
print(len(location_for_target_0))
location_for_target_0[:20]

2143


[nan,
 'Est. September 2012 - Bristol',
 'Philadelphia, PA',
 'London, UK',
 'Pretoria',
 'Paranaque City',
 'Live On Webcam',
 'milky way',
 'England.',
 'Barbados',
 'Abuja',
 'Sao Paulo, Brazil',
 'hollywoodland ',
 'Inang Pamantasan',
 'Twitter Lockout in progress',
 'Calgary, AB',
 'San Francisco',
 'CLVLND',
 'UK',
 'St. Louis, MO']

In [28]:
location_for_target_0 = list(map(str,location_for_target_0))
location_for_target_0.sort()
location_for_target_0[:20]

['  Glasgow ',
 '  Melbourne, Australia',
 '  å_ ',
 " 45å¡ 5'12.53N   14å¡ 7'24.93E",
 ' ?currently writing a book?',
 ' Alex/Mika/Leo|18|he/she/they',
 ' BC, US, Asia or Europe.',
 ' Blood Indian Reserve',
 ' Eugene, Oregon',
 ' Indiana',
 ' Jariana Town',
 ' Little Rock, AR',
 ' Miami Beach',
 ' Neverland ',
 ' New Delhi ',
 ' Nxgerxa',
 ' Quantico Marine Base, VA.',
 ' Queensland, Australia',
 ' Road to the Billionaires Club',
 ' Somewhere.']

### Catch all possible keyword for target_0 in train data

In [29]:
keyword_for_target_0 = target_0.keyword.unique()
keyword_for_target_0 = keyword_for_target_0.tolist()
print(len(keyword_for_target_0))
keyword_for_target_0[:20]

219


[nan,
 'ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze']

In [30]:
keyword_for_target_0 = list(map(str,keyword_for_target_0))
keyword_for_target_0.sort()
keyword_for_target_0[:20]

['ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing']

* **Note** we note that keyword is not a real good classifier to obtain our target, and also we note that location can be an important feature to add with the text to predict our target.

## ToDo:
1. How to replace missing values?
2. Put the words (location, keyword) in the best format. For example for this location "45å¡ 5'12.53N   14å¡ 7'24.93E" => ???
3. With which features (columns), we will fit our model?


# Preprocessing data

## Missing values

In [5]:
train_data= train_data.dropna(subset=['location'])
train_data

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [10]:
import preprocess.location as pl
from geopy.geocoders import Nominatim
from typing import Tuple,Any
featuretest_data= train_data.copy()

class ConversionProgress(pl.GeolocationStepConverter):
    def __init__(self,frequency:int):
        self.frequency=frequency
        self.counter=0
        self.progress=0
    def convert(self, address:str)-> Tuple[Any,bool]:
        self.progress+=1
        self.counter+=1
        if self.counter >= self.frequency:
            self.counter=0
            print(f"Current progress: {self.progress}")
        return address,False    
        


In [19]:
geoLoc = Nominatim(user_agent="GetLoc")
rateLimited = pl.RateLimitedGeocoder(geoLoc, min_delay_seconds=1)
layer=pl.DefaultPreprocessLocationLayer(force_update=False)
layer.transformers.append(ConversionProgress(frequency=100))
layer.transformers.append(pl.StripLocation())
layer.transformers.append(pl.LatLongToGeolocation(rateLimited))
layer.transformers.append(pl.AddressToGeolocation(rateLimited))
featuretest_data=layer.preprocess(featuretest_data)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Livingston, IL  U.S.A.',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 1390, in getresponse
    response.begin()
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 286, in _read_status
    line = str

Current progress: 100
Current progress: 200


RateLimiter caught an error, retrying (0/2 tries). Called with (*('?s????ss? a?????',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 1390, in getresponse
    response.begin()
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 286, in _read_status
    line = str(self.

Current progress: 300


KeyboardInterrupt: 

In [16]:
featuretest_data.to_pickle("location.pkl")

In [20]:
featuretest_data=pd.read_pickle("location.pkl")

In [21]:
featuretest_data["

,id,keyword,location,text,target,geolocation,geolocation_converted
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,"(Birmingham, West Midlands Combined Authority,...",True
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,NaN,False
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,"(Africa, (11.5024338, 17.7578122))",True
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,"(Philadelphia, Philadelphia County, Pennsylvan...",True
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"(London, Greater London, England, United Kingd...",True
...,...,...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0,"(تونس, (33.8439408, 9.400138))",True
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0,NaN,False
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,"(Vancouver, Metro Vancouver Regional District,...",True
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,"(London, Greater London, England, United Kingd...",True
